In [1]:
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from skimage.transform import resize
from skimage.io import imread
from skimage import img_as_ubyte
import numpy as np

Using TensorFlow backend.


In [ ]:
model.load_weights('my_save_file.h5')

In [3]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model


In [3]:
im = imread('examples/dog.jpg')
im = resize(im, (224, 224))
im = img_as_ubyte(im)
im = im.astype(np.float32)
im[:,:,0] -= 103.939
im[:,:,1] -= 116.779
im[:,:,2] -= 123.68
im = im.transpose((2,0,1))
im = np.expand_dims(im, axis=0)

/Users/Chris/anaconda/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [4]:
def preprocess_image_batch(image_paths, img_size=None, crop_size=None, color_mode="rgb", out=None):
    img_list = []
    
    for im_path in image_paths:
        img = imread(im_path, mode='RGB')
        if img_size:
            img = imresize(img,img_size)
            
        img = img.astype('float32')
        # We permute the colors to get them in the BGR order
        if color_mode=="bgr":
            img[:,:,[0,1,2]] = img[:,:,[2,1,0]]
        # We normalize the colors with the empirical means on the training set
        img[:, :, 0] -= 123.68 
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 103.939
        img = img.transpose((2, 0, 1))

        if crop_size:
            img = img[:,(img_size[0]-crop_size[0])//2:(img_size[0]+crop_size[0])//2
                      ,(img_size[1]-crop_size[1])//2:(img_size[1]+crop_size[1])//2]
            
        img_list.append(img)

    img_batch = np.stack(img_list, axis=0)
    if not out is None:
        out.append(img_batch)
    else:
        return img_batch

In [44]:
from scipy.misc import imread, imresize, imsave
im = preprocess_image_batch(['examples/dog3.jpg'],img_size=(256,256), crop_size=(224,224), color_mode="bgr")

In [34]:
# Create Model
model = VGG_16()

In [35]:
# Load model weights
# WARNING - This is ~550MB of weights, be patient
model.load_weights('weights/vgg16_weights.h5')

In [37]:
#convert layers between Theano and TensorFlow Convolution
from keras import backend as K
from keras.utils.np_utils import convert_kernel
import tensorflow as tf

In [38]:
# This is from Theano ---> TensorFlow
ops = []
for layer in model.layers:
   if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D']:
      original_w = K.get_value(layer.W)
      converted_w = convert_kernel(original_w)
      ops.append(tf.assign(layer.W, converted_w).op)

In [39]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

In [40]:
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [46]:
# result for 'dog.jpg' - 353
# result for 'dog2.jpg' - 159
# result for 'dog3.jpg' - 225
# 
out = model.predict(im)
max_out = np.argmax(out)
print max_out

225


In [32]:
from os import listdir
from os.path import isfile, join, dirname

from scipy.io import loadmat



meta_clsloc_file = ('data/meta_clsloc.mat')


synsets = loadmat(meta_clsloc_file)["synsets"][0]

synsets_imagenet_sorted = sorted([(int(s[0]), str(s[1][0])) for s in synsets[:1000]],
                                 key=lambda v:v[1])

corr = {}
for j in range(1000):
    corr[synsets_imagenet_sorted[j][0]] = j

corr_inv = {}
for j in range(1,1001):
    corr_inv[corr[j]] = j

def depthfirstsearch(id, out=None):
    if out == None:
        out = []
    if type(id) == int:
        pass
    else:
        id = next(int(s[0]) for s in synsets if s[1][0] == id)
        
    out.append(id)
    children = synsets[id-1][5][0]
    for c in children:
        depthfirstsearch(int(c), out)
    return out
    
def synset_to_dfs_ids(synset):
    ids = [x for x in depthfirstsearch(synset) if x <= 1000]
    ids = [corr[x] for x in ids]
    return ids
    

def synset_to_id(synset):
    a = next((i for (i,s) in synsets if s == synset), None)
    return a


def id_to_synset(id):
    return str(synsets[corr_inv[id]-1][1][0])
    

def id_to_words(id):
    return synsets[corr_inv[id]-1][2][0]

def pprint_output(out, n_max_synsets=10):
    best_ids = out.argsort()[::-1][:10]
    for u in best_ids:
        print("%.2f"% round(100*out[u],2)+" : "+id_to_words(u))

In [33]:
print synset_to_id(max_out)

ValueError: too many values to unpack